In [39]:
from paddleocr import PaddleOCR,draw_ocr
import cv2
from PIL import Image, ImageFont
import numpy as np
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
img_path = '../destijl_dataset/04_text/0007.png'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)


result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
image = cv2.imread(img_path)

for bbox in boxes:
    x, y = bbox[0][0], bbox[0][1]
    z, t = bbox[2][0], bbox[2][1]
    cv2.rectangle(image, (int(x), int(y)), (int(z), int(t)), (0, 255, 0), 2)
    cropped_image = image[y:t, x:z]
    
cv2.imwrite('result.jpg', image)

pixels = np.float32(image.reshape(-1, 3))

n_colors = 4
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
palette = np.asarray(palette, dtype=np.int64)
palette_w_white = []

for color in palette:
    x, y, z = color
    if not (252 < x < 256 and 252 < y < 256 and 252 < z < 256):
        palette_w_white.append(color)

_, counts = np.unique(labels, return_counts=True)
dominant = palette_w_white[np.argmax(counts)]


[2023/06/27 11:41:04] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/busraasan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/busraasan/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.

TypeError: slice indices must be integers or None or have an __index__ method

In [48]:
from paddleocr import PaddleOCR,draw_ocr
import cv2
from PIL import Image, ImageFont
import numpy as np
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
img_path = '../destijl_dataset/04_text/0039.png'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
image = cv2.imread(img_path)

n_colors = 3
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

county = 0
for bbox in boxes:
    x, y = int(bbox[0][0]), int(bbox[0][1])
    z, t = int(bbox[2][0]), int(bbox[2][1])
    #cv2.rectangle(image, (x, y), (z, t), (0, 255, 0), 2)
    cropped_image = image[y:t, x:z]
    pixels = np.float32(cropped_image.reshape(-1, 3))

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    palette = np.asarray(palette, dtype=np.int64)
    palette_w_white = []

    for i, color in enumerate(palette):
        x, y, z = color
        if not (252 < x < 256 and 252 < y < 256 and 252 < z < 256):
            palette_w_white.append(color)
        else:
            labels = np.delete(labels, np.where(labels == i))

    _, counts = np.unique(labels, return_counts=True)
    dominant = palette_w_white[np.argmax(counts)]
    
    if county == 3:
        print("Printing dominant color for bbox")
        print("COUNT")
        print(counts)
        print("PALETTE")
        print(palette_w_white)
        print("DOMINANT")
        print(dominant)
    county += 1

[2023/06/27 12:04:52] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/busraasan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/busraasan/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.

In [41]:
counts

array([5483,  995])